In [1]:
# Install required python packages
!pip install pandas==1.3.5
!pip install xport==2.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 30.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.9.0 requires pandas>=1.5.0, but you have pandas 1.3.5 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.3.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.3.5 which is incompatible.
plotnine 0.12.4 requires pandas>=1.5.0, but you have pandas 1.3.5 which is incompatible.
statsmodels 0.14.2 requires pandas!=2.1.0,>=1.4, but you have pandas 1.3.5 which is incompatible.
xarray 2023.7.0 requires pandas>=1.4, but you have pandas 1.3.5 which is incompatible.


In [2]:
# Import required python modules
import pandas as pd
import xport
import numpy as np

In [3]:
# Read demographic raw SAS dataset
demog = pd.read_sas('/content/demog.sas7bdat')
# First five observations from the dataset
demog.head()

,Subject,SiteNumber,AGE,AGEU,sexc,ENHISP_RAW,EOHISP_RAW,RWHITE_RAW,RBLACK_RAW,RAIAN_RAW,RINDIA_RAW,RNREP_RAW,RUNK_RAW,ROTHER_RAW
0,b'0802-102',b'0802',75.0,b'YEARS',b'Male',b'1',b'0',b'0',b'0',b'0',b'1',b'0',b'0',b'0'
1,b'1661-101',b'1661',64.0,b'YEARS',b'Male',b'1',b'0',b'0',b'0',b'0',b'1',b'0',b'0',b'0'
2,b'0210-101',b'0210',88.0,b'YEARS',b'Male',b'1',b'0',b'0',b'0',b'0',b'1',b'0',b'0',b'0'
3,b'0210-102',b'0210',74.0,b'YEARS',b'Male',b'1',b'0',b'0',b'0',b'0',b'1',b'0',b'0',b'0'
4,b'0802-104',b'0802',58.0,b'YEARS',b'Male',b'1',b'0',b'0',b'0',b'0',b'1',b'0',b'0',b'0'


In [4]:
# Remove special characters from dataset
demog = pd.read_sas('/content/demog.sas7bdat', encoding = 'latin-1')
demog.head()

,Subject,SiteNumber,AGE,AGEU,sexc,ENHISP_RAW,EOHISP_RAW,RWHITE_RAW,RBLACK_RAW,RAIAN_RAW,RINDIA_RAW,RNREP_RAW,RUNK_RAW,ROTHER_RAW
0,0802-102,0802,75.0,YEARS,Male,1,0,0,0,0,1,0,0,0
1,1661-101,1661,64.0,YEARS,Male,1,0,0,0,0,1,0,0,0
2,0210-101,0210,88.0,YEARS,Male,1,0,0,0,0,1,0,0,0
3,0210-102,0210,74.0,YEARS,Male,1,0,0,0,0,1,0,0,0
4,0802-104,0802,58.0,YEARS,Male,1,0,0,0,0,1,0,0,0


In [5]:
# Create DOMAIN, STUDYID and COUNTRY varaibles
demog['DOMAIN'], demog['STUDYID'], demog['COUNTRY'] = 'DM', 'ABC-PROJECT-001', 'INDIA'
# Create SEX variable
demog['SEX'] = demog.sexc.replace(['Male','Female'], ['M','F'])
# Create SUBJID and SITEID variables
dm1 = demog.rename(columns={'Subject':'SUBJID', 'SiteNumber':'SITEID'})
# Create USUBJID variable
dm1['USUBJID'] = dm1.STUDYID + '-' + dm1.SUBJID
dm1.head()

,SUBJID,SITEID,AGE,AGEU,sexc,ENHISP_RAW,EOHISP_RAW,RWHITE_RAW,RBLACK_RAW,RAIAN_RAW,RINDIA_RAW,RNREP_RAW,RUNK_RAW,ROTHER_RAW,DOMAIN,STUDYID,COUNTRY,SEX,USUBJID
0,0802-102,0802,75.0,YEARS,Male,1,0,0,0,0,1,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0802-102
1,1661-101,1661,64.0,YEARS,Male,1,0,0,0,0,1,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-1661-101
2,0210-101,0210,88.0,YEARS,Male,1,0,0,0,0,1,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-101
3,0210-102,0210,74.0,YEARS,Male,1,0,0,0,0,1,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-102
4,0802-104,0802,58.0,YEARS,Male,1,0,0,0,0,1,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0802-104


In [6]:
for index, _dm2 in dm1.iterrows():
## Create RACE
  if _dm2['RAIAN_RAW'].__contains__('1'):
     dm1.loc[index, 'RACE'] = 'AMERICAN INDIAN OR ALASKA NATIVE'
  elif _dm2['RINDIA_RAW'].__contains__('1'):
     dm1.loc[index, 'RACE'] = 'ASIAN'
  elif _dm2['RBLACK_RAW'].__contains__('1'):
     dm1.loc[index, 'RACE'] = 'BLACK OR AFRICAN AMERICAN'
  elif _dm2['RWHITE_RAW'].__contains__('1'):
     dm1.loc[index, 'RACE'] = 'WHITE'
  elif _dm2['RNREP_RAW'].__contains__('1'):
     dm1.loc[index, 'RACE'] = 'NOT REPORTED'
  elif _dm2['ROTHER_RAW'].__contains__('1'):
     dm1.loc[index, 'RACE'] = 'OTHER'
  elif _dm2['RUNK_RAW'].__contains__('1'):
     dm1.loc[index, 'RACE'] = 'UNKNOWN'

## Create ETHNIC
  if _dm2['ENHISP_RAW'].__contains__('1'):
     dm1.loc[index, 'ETHNIC'] = 'NOT HISPANIC OR LATINO'
  else:
     dm1.loc[index, 'ETHNIC'] = 'HISPANIC OR LATINO'

dm1

,SUBJID,SITEID,AGE,AGEU,sexc,ENHISP_RAW,EOHISP_RAW,RWHITE_RAW,RBLACK_RAW,RAIAN_RAW,...,RNREP_RAW,RUNK_RAW,ROTHER_RAW,DOMAIN,STUDYID,COUNTRY,SEX,USUBJID,RACE,ETHNIC
0,0802-102,0802,75.0,YEARS,Male,1,0,0,0,0,...,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0802-102,ASIAN,NOT HISPANIC OR LATINO
1,1661-101,1661,64.0,YEARS,Male,1,0,0,0,0,...,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-1661-101,ASIAN,NOT HISPANIC OR LATINO
2,0210-101,0210,88.0,YEARS,Male,1,0,0,0,0,...,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-101,ASIAN,NOT HISPANIC OR LATINO
3,0210-102,0210,74.0,YEARS,Male,1,0,0,0,0,...,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-102,ASIAN,NOT HISPANIC OR LATINO
4,0802-104,0802,58.0,YEARS,Male,1,0,0,0,0,...,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0802-104,ASIAN,NOT HISPANIC OR LATINO
5,1667-101,1667,72.0,YEARS,Male,1,0,0,0,0,...,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-1667-101,ASIAN,NOT HISPANIC OR LATINO
6,0327-101,0327,77.0,YEARS,Female,1,0,0,0,0,...,0,0,0,DM,ABC-PROJECT-001,INDIA,F,ABC-PROJECT-001-0327-101,ASIAN,NOT HISPANIC OR LATINO
7,0210-103,0210,83.0,YEARS,Male,1,0,0,0,0,...,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-103,ASIAN,NOT HISPANIC OR LATINO
8,0210-104,0210,81.0,YEARS,Female,1,0,0,0,0,...,0,0,0,DM,ABC-PROJECT-001,INDIA,F,ABC-PROJECT-001-0210-104,ASIAN,NOT HISPANIC OR LATINO
9,0782-101,0782,73.0,YEARS,Male,1,0,0,0,0,...,0,0,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0782-101,ASIAN,NOT HISPANIC OR LATINO


In [7]:
# Read exposure raw SAS dataset
ex = pd.read_sas('/content/exp.sas7bdat', encoding = 'latin-1')
ex.head()

,Subject,ex_date
0,0210-101,2021-01-13
1,0210-101,2021-01-27
2,0210-101,2021-02-10
3,0210-101,2021-04-07
4,0210-101,2021-07-07


In [8]:
### Find the first and last exposure date
# sort by subject and exposure dates
ex2 = ex.sort_values(by =['Subject','ex_date'], ascending=[True, True] )
# pick the first exposure date of the subject
ex3 = ex2.groupby('Subject').first()
# reset index so that subject become a variable.
ex4 = ex3.reset_index()
ex4

,Subject,ex_date
0,0210-101,2021-01-13
1,0210-102,2021-01-13
2,0210-103,2021-02-10
3,0210-105,2021-02-10
4,0210-106,2021-05-12
5,0327-101,2021-01-28
6,0327-102,2021-03-12
7,0327-103,2021-03-05
8,0782-101,2021-03-10
9,0802-101,2020-11-25


In [9]:
## Select and rename the variables
df_ex4 = ex4[['Subject','ex_date']].rename(columns={'Subject':'SUBJID','ex_date':'RFSTDTC'})
df_ex4.head()

,SUBJID,RFSTDTC
0,0210-101,2021-01-13
1,0210-102,2021-01-13
2,0210-103,2021-02-10
3,0210-105,2021-02-10
4,0210-106,2021-05-12


In [10]:
# pick the last exposure date of the subject
ex5 = ex2.groupby('Subject').last()
# reset index so that subject become a variable.
ex5 = ex5.reset_index()
ex5

,Subject,ex_date
0,0210-101,2023-07-12
1,0210-102,2021-01-13
2,0210-103,2023-08-16
3,0210-105,2023-08-16
4,0210-106,2023-11-08
5,0327-101,2023-07-27
6,0327-102,2023-09-07
7,0327-103,2023-08-31
8,0782-101,2023-09-13
9,0802-101,2021-11-18


In [11]:
# Select and rename variables
df_ex5 = ex5[['Subject','ex_date']].rename(columns={'Subject':'SUBJID','ex_date':'RFENDTC'})
df_ex5.head()

,SUBJID,RFENDTC
0,0210-101,2023-07-12
1,0210-102,2021-01-13
2,0210-103,2023-08-16
3,0210-105,2023-08-16
4,0210-106,2023-11-08


In [12]:
### merge first and last exposure date
df_ex6 = pd.merge(df_ex4, df_ex5, on='SUBJID')
df_ex6

,SUBJID,RFSTDTC,RFENDTC
0,0210-101,2021-01-13,2023-07-12
1,0210-102,2021-01-13,2021-01-13
2,0210-103,2021-02-10,2023-08-16
3,0210-105,2021-02-10,2023-08-16
4,0210-106,2021-05-12,2023-11-08
5,0327-101,2021-01-28,2023-07-27
6,0327-102,2021-03-12,2023-09-07
7,0327-103,2021-03-05,2023-08-31
8,0782-101,2021-03-10,2023-09-13
9,0802-101,2020-11-25,2021-11-18


In [13]:
### merge exposure data to demog data
# sort by subject
dm2 = dm1.sort_values(by =['SUBJID'], ascending=[True] )
# merge exposure information to dm
df_dm3 = pd.merge(dm2, df_ex6, on='SUBJID', how='left')
df_dm3

,SUBJID,SITEID,AGE,AGEU,sexc,ENHISP_RAW,EOHISP_RAW,RWHITE_RAW,RBLACK_RAW,RAIAN_RAW,...,ROTHER_RAW,DOMAIN,STUDYID,COUNTRY,SEX,USUBJID,RACE,ETHNIC,RFSTDTC,RFENDTC
0,0210-101,0210,88.0,YEARS,Male,1,0,0,0,0,...,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-101,ASIAN,NOT HISPANIC OR LATINO,2021-01-13,2023-07-12
1,0210-102,0210,74.0,YEARS,Male,1,0,0,0,0,...,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-102,ASIAN,NOT HISPANIC OR LATINO,2021-01-13,2021-01-13
2,0210-103,0210,83.0,YEARS,Male,1,0,0,0,0,...,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-103,ASIAN,NOT HISPANIC OR LATINO,2021-02-10,2023-08-16
3,0210-104,0210,81.0,YEARS,Female,1,0,0,0,0,...,0,DM,ABC-PROJECT-001,INDIA,F,ABC-PROJECT-001-0210-104,ASIAN,NOT HISPANIC OR LATINO,NaN,NaN
4,0210-105,0210,77.0,YEARS,Male,1,0,0,0,0,...,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-105,ASIAN,NOT HISPANIC OR LATINO,2021-02-10,2023-08-16
5,0210-106,0210,75.0,YEARS,Male,1,0,0,0,0,...,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-106,ASIAN,NOT HISPANIC OR LATINO,2021-05-12,2023-11-08
6,0327-101,0327,77.0,YEARS,Female,1,0,0,0,0,...,0,DM,ABC-PROJECT-001,INDIA,F,ABC-PROJECT-001-0327-101,ASIAN,NOT HISPANIC OR LATINO,2021-01-28,2023-07-27
7,0327-102,0327,82.0,YEARS,Male,1,0,0,0,0,...,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0327-102,ASIAN,NOT HISPANIC OR LATINO,2021-03-12,2023-09-07
8,0327-103,0327,75.0,YEARS,Female,1,0,0,0,0,...,0,DM,ABC-PROJECT-001,INDIA,F,ABC-PROJECT-001-0327-103,ASIAN,NOT HISPANIC OR LATINO,2021-03-05,2023-08-31
9,0782-101,0782,73.0,YEARS,Male,1,0,0,0,0,...,0,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0782-101,ASIAN,NOT HISPANIC OR LATINO,2021-03-10,2023-09-13


In [14]:
### Prepare demographic data
# Drop variables
df_dm4 = df_dm3.drop(columns=['sexc','RINDIA_RAW','RAIAN_RAW','RBLACK_RAW','RWHITE_RAW','RNREP_RAW','ROTHER_RAW','RUNK_RAW','ENHISP_RAW','EOHISP_RAW'])
df_dm4

,SUBJID,SITEID,AGE,AGEU,DOMAIN,STUDYID,COUNTRY,SEX,USUBJID,RACE,ETHNIC,RFSTDTC,RFENDTC
0,0210-101,0210,88.0,YEARS,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-101,ASIAN,NOT HISPANIC OR LATINO,2021-01-13,2023-07-12
1,0210-102,0210,74.0,YEARS,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-102,ASIAN,NOT HISPANIC OR LATINO,2021-01-13,2021-01-13
2,0210-103,0210,83.0,YEARS,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-103,ASIAN,NOT HISPANIC OR LATINO,2021-02-10,2023-08-16
3,0210-104,0210,81.0,YEARS,DM,ABC-PROJECT-001,INDIA,F,ABC-PROJECT-001-0210-104,ASIAN,NOT HISPANIC OR LATINO,NaN,NaN
4,0210-105,0210,77.0,YEARS,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-105,ASIAN,NOT HISPANIC OR LATINO,2021-02-10,2023-08-16
5,0210-106,0210,75.0,YEARS,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0210-106,ASIAN,NOT HISPANIC OR LATINO,2021-05-12,2023-11-08
6,0327-101,0327,77.0,YEARS,DM,ABC-PROJECT-001,INDIA,F,ABC-PROJECT-001-0327-101,ASIAN,NOT HISPANIC OR LATINO,2021-01-28,2023-07-27
7,0327-102,0327,82.0,YEARS,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0327-102,ASIAN,NOT HISPANIC OR LATINO,2021-03-12,2023-09-07
8,0327-103,0327,75.0,YEARS,DM,ABC-PROJECT-001,INDIA,F,ABC-PROJECT-001-0327-103,ASIAN,NOT HISPANIC OR LATINO,2021-03-05,2023-08-31
9,0782-101,0782,73.0,YEARS,DM,ABC-PROJECT-001,INDIA,M,ABC-PROJECT-001-0782-101,ASIAN,NOT HISPANIC OR LATINO,2021-03-10,2023-09-13


In [15]:
# Save demographic SDTM data in XPT format
with open('/content/cdisc/dm.xpt', 'wb') as f:
  xport.from_dataframe(df_dm4, f)